In [10]:
from src.uk_ner_hf_spacy_wrapper import build_spacy_wrapper

In [11]:
FINETUNED_PATH = "models/my_finetuned_ner"
SPACY_MODEL_NAME = "models/uk_spacy_model"
TEST_DATA_PATH = "data/test_dataset.json"

In [13]:
import spacy
import json
from spacy.training import offsets_to_biluo_tags
from seqeval.metrics import classification_report, f1_score

print(f"--- 1. Building spaCy model wrapper '{SPACY_MODEL_NAME}' ---")
build_spacy_wrapper(SPACY_MODEL_NAME, FINETUNED_PATH)

# КРОК 2: Тестування (Inference)

nlp_test = spacy.load(SPACY_MODEL_NAME)


with open(TEST_DATA_PATH, 'r', encoding='utf-8') as f:
    data = json.load(f)

true_labels = []
pred_labels = []

for item in data:
    text = item['text']
    gold_ents = item.get('entities', [])

    # Еталонні теги (Gold)
    doc_gold = nlp_test.make_doc(text)
    tags = offsets_to_biluo_tags(doc_gold, gold_ents)
    gold_iob = [t.replace('U-', 'B-').replace('L-', 'I-') if t != '-' else 'O' for t in tags]

    doc_pred = nlp_test(text)
    pred_iob = [f"{token.ent_iob_}-{token.ent_type_}" if token.ent_iob_ != 'O' else 'O' for token in doc_pred]

    true_labels.append(gold_iob)
    pred_labels.append(pred_iob)

    if gold_iob != pred_iob and any(t != 'O' for t in gold_iob) and any(t != 'O' for t in pred_iob):
        print(f"\nTEXT: {text}")
        print(f"GOLD: {gold_iob}")
        print(f"PRED: {pred_iob}")

report = classification_report(true_labels, pred_labels, output_dict=True, zero_division=0)
overall_f1 = f1_score(true_labels, pred_labels, zero_division=0)

gpe_recall = report.get('GPE', {}).get('recall', 0.0)
loc_recall = report.get('LOC', {}).get('recall', 0.0)

print("\n" + "="*40)
print(f"PURE MODEL METRICS (NO HARDCODED RULES)")
print("="*40)
print(f"Overall F1 Score:  {overall_f1:.4f}")
print(f"GPE Recall: {gpe_recall:.4f}")
print(f"LOC Recall: {loc_recall:.4f}")
print("-" * 40)

--- 1. Building spaCy model wrapper 'models/uk_spacy_model' ---
Directory models/uk_spacy_model already exists. Skipping creation.
Loading HF model from: models/my_finetuned_ner on device -1...


Device set to use cpu
/usr/local/lib/python3.11/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Селище Гладосове розташоване неподалік від лінії р..." with entities "[[7, 16, 'GPE'], [60, 77, 'LOC']]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



--- ERROR ANALYSIS (Where Model Disagrees with Data) ---


/usr/local/lib/python3.11/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Село Ревне знаходиться у Бориспільському районі, щ..." with entities "[[5, 10, 'GPE'], [25, 45, 'LOC'], [53, 61, 'LOC']]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(



TEXT: Через село Грушівка протікає річка Чортомлик, впадаючи у дніпровські плавні.
GOLD: ['O', 'O', 'B-GPE', 'O', 'B-LOC', 'I-LOC', 'O', 'O', 'O', 'O', 'O', 'O']
PRED: ['O', 'O', 'B-GPE', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

TEXT: Новогригорівка є частиною Миколаївської області, відомої своїми степами.
GOLD: ['B-GPE', 'O', 'O', 'B-LOC', 'I-LOC', 'O', 'O', 'O', 'O', 'O']
PRED: ['B-GPE', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'O']

TEXT: Оришківці лежать у мальовничій долині річки Серет.
GOLD: ['B-GPE', 'O', 'O', 'O', 'O', 'B-LOC', 'I-LOC', 'O']
PRED: ['B-GPE', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'O']

TEXT: Поворська громада активно співпрацює з Ковельським районом.
GOLD: ['B-GPE', 'I-GPE', 'O', 'O', 'O', 'B-LOC', 'I-LOC', 'O']
PRED: ['B-GPE', 'B-GPE', 'O', 'O', 'O', 'B-LOC', 'O', 'O']

TEXT: Устечко вважається одним із найкрасивіших місць Дністровського каньйону.
GOLD: ['B-GPE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
PRED: ['B-GPE', 'O', 'O', 'O', 'O', 'O', 'B-GPE', '

/usr/local/lib/python3.11/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Устечко вважається одним із найкрасивіших місць Дн..." with entities "[[0, 7, 'GPE'], [40, 63, 'LOC']]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
